In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/vehicle-dataset-from-cardekho/Car details v3.csv')
df

In [ ]:
df.head()

In [ ]:
# shape of data
df.shape

In [ ]:
# missing values
df.isnull().sum()

In [ ]:
# info
df.info()

In [ ]:
# duplicate data
df.duplicated().sum()

In [ ]:
# describe function
df.describe()

In [ ]:
4.500000e+05

In [ ]:
# Observation
# 1. Missing values in some cols
# 2. Seats is float should be int
# 3. more than 1000 rows are duplicates
# 4. Outliers in year,selling price,seats
# 5. Torque,engine,mileage and max_power have unnecceary units


In [ ]:
# drop any row with missing values
df.dropna(inplace=True)#doing inplace = True for permanent changes

In [ ]:
df.shape

In [ ]:
# no of dropped rows
8128 - 7906
df.isnull().sum()

In [ ]:
# remove duplicate rows
df = df.drop_duplicates(keep='first')#keep first mean only kepting first row of it's same kind of 
#duplicat rows

In [ ]:
df.shape

In [ ]:
df.duplicated().sum()

In [ ]:
# change data type of seats col
df['seats'] = df['seats'].astype('int32')

In [ ]:
df.info()

In [ ]:
# Handling mileage col
df['mileage'] = df['mileage'].str.split(expand=True)[0]
df['mileage'] = df['mileage'].astype('float64')

In [ ]:
# Handling engine col
df['engine'] = df['engine'].str.split(expand=True)[0]
df['engine'] = df['engine'].astype('int32')

In [ ]:
# Handling max_power col
df['max_power'] = df['max_power'].str.split(expand=True)[0]
df['max_power'] = df['max_power'].astype('float64')

In [ ]:
df.head()

In [ ]:
# dropping the torque col
df.drop(columns=['torque'],inplace=True)

In [ ]:
df.head()

In [ ]:
#to find the catagory in the name column
df['name'].unique().shape

In [ ]:
df['name'].str.split(expand=True)

In [ ]:
# creating a new column brand and extreacting the value from name to brand
df['brand'] = df['name'].str.split(expand=True)[0]

In [ ]:
df.drop(columns=['name'],inplace=True)
df.head()

In [ ]:
df['brand'].value_counts()

In [ ]:
freq_brands = df['brand'].value_counts()[df['brand'].value_counts()>100].index.tolist()

In [ ]:
df = df[df['brand'].isin(freq_brands)]

In [ ]:
df.head()

In [ ]:
freq_fuel = ['Diesel','Petrol']
df = df[df['fuel'].isin(freq_fuel)]

In [ ]:
df['seller_type'].value_counts()

In [ ]:
df = df[df['seller_type'].isin(['Individual','Dealer'])]

In [ ]:
df = df[df['owner'].isin(['First Owner','Second Owner','Third Owner'])]

In [ ]:
df.shape

In [ ]:
import seaborn as sns
sns.boxplot(df['km_driven'])

In [ ]:
q1 = df['km_driven'].quantile(0.25)
q3 = df['km_driven'].quantile(0.75)
iqr = q3 - q1

In [ ]:
max_val_km_driven = q3 + 1.5*iqr

In [ ]:
max_val_km_driven

In [ ]:
df.head()

In [ ]:
df['km_driven'] = np.where(df['km_driven']>max_val_km_driven,max_val_km_driven,df['km_driven'])
#by where checking where the conditions is true and where the condition is true first write the condition for being true 
#and second write the conditon for false

In [ ]:
sns.boxplot(df['year'])

In [ ]:
df = df[df['year'] >= 2000]

In [ ]:
sns.boxplot(df['mileage'])

In [ ]:
mean_mileage = df[df['mileage'] !=0]['mileage'].median()
mean_mileage

In [ ]:
df['mileage'] = np.where(df['mileage'] == 0,mean_mileage,df['mileage'])

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
X = df.drop(columns=['selling_price'])
y = df['selling_price']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)


In [ ]:
# Ordinal encoding on Owner col
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
tnf = ColumnTransformer(
    [
        ('ordinal',OrdinalEncoder(),['owner']),
        ('nominal',OneHotEncoder(drop='first',sparse=False),['fuel','seller_type','transmission','brand'])
    ], remainder='passthrough'
)

In [ ]:
X_train_tnf = tnf.fit_transform(X_train)
X_test_tnf = tnf.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()


In [ ]:
lr.fit(X_train_tnf,y_train)


In [ ]:
y_pred = lr.predict(X_test_tnf)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)

In [ ]:
X_train_poly = poly.fit_transform(X_train_tnf)
X_test_poly = poly.transform(X_test_tnf)

In [ ]:
lr = LinearRegression()
lr.fit(X_train_poly,y_train)
y_pred = lr.predict(X_test_poly)
r2_score(y_test,y_pred)

In [ ]:
#pipleline complete upper set into one code
from sklearn.pipeline import Pipeline

In [ ]:
 pipe=Pipeline([
     ('column_transformer',tnf),
     ('polynomial',poly),
     ('linear_regression',lr)]
 )

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
y_pred=pipe.predict(X_test)

In [ ]:
r2_score(y_test,y_pred)

In [ ]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))